# Creating the Boundary Conditions

This notebook serves as a demo for the creation of model boundary conditions. These boundary conditions are shown for Mike's Calfornia Current regional model. You can follow and/or adapt this notebook based on your configuration if you are using a regional domain.

First, import packages to re-create and visualize the model fields here:

In [42]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import netCDF4 as nc4

Next, define the location of the input directory for the model. This is the same directory that holds the bathymetry file generated in the previous notebooks for this model example.

In [43]:
# define the input directory
input_dir = os.getcwd()

## Constructing the Boundary Conditions
For my model, I will use a model state from the ECCO Version 5 state estimate. I will prepare the boundary condition fields in 7 steps:
1. download 5 fields and 4 grid files generated by the ECCO model in 2008
2. read the ECCO model grid
3. read in the bathymetry for my model as well as its grid
4. prepare the ECCO fields for interpolation
5. interpolate the ECCO fields onto my model grid and store each as a binary file
6. plot the interpolated fields to ensure they look as expected
7. prepare notes on the run-time options I will use to implement my approach

### Step 1: Download the ECCO fields
To begin, I downloaded the model fields generated by the ECCO Version 4 Alpha state estimate. These fields are available [HERE](https://podaac.jpl.nasa.gov/ECCO?tab=mission-objectives&sections=about%2Bdata). In particular, I downloaded the following list of files that contain the field pertaining to my model (2008):

| Variable(s) | Location |
| -------- | --------- |
|THETA and SALT|[2008](https://cmr.earthdata.nasa.gov/virtual-directory/collections/C1991543728-POCLOUD/temporal/2008)|
|UVEL and VVEL|[2008](https://cmr.earthdata.nasa.gov/virtual-directory/collections/C1991543728-POCLOUD/temporal/2008/12)|

I also store the year as a variable:

In [44]:
year = 1997

I stored these fields in the same directory as this notebook:

In [45]:
# define the input directory
input_dir = os.getcwd()
# data_folder = os.getcwd()
data_folder = '/Users/cool/Workspace/MLML/ms274_fall2025/Project/notebook_input'

### Step 2: Read in the ECCO grid
To read in the ECCO fields, I will rely on the `grid` module from the `eccoseas.ecco` package [HERE](https://github.com/mhwood/eccoseas), which I import here:

In [46]:
from eccoseas.ecco import grid

In [47]:
ds = nc4.Dataset('../../notebook_input/GRID_GEOMETRY_ECCO_V4r4_native_llc0090.nc')
ecco_XC_tiles = ds.variables['XC'][:,:,:]
ecco_YC_tiles = ds.variables['YC'][:,:,:]
ecco_hFacC_tiles = ds.variables['hFacC'][:,:,:,:]
ecco_hFacS_tiles = ds.variables['hFacS'][:,:,:,:]
ecco_hFacW_tiles = ds.variables['hFacW'][:,:,:,:]
ecco_RF_tiles = ds.variables['Z'][:]
AngleCS = ds.variables['CS'][:, :, :]
AngleSN = ds.variables['SN'][:, :, :]
ds.close()

As described [HERE](https://ecco-v4-python-tutorial.readthedocs.io/ECCO_v4_Plotting_Tiles.html), the ECCO grid has 13 tiles but only 1 or 2 may pertain to my local area. To determine which tiles correspond to my region, I'll read in my model grid next.

### Step 3: Read in the Model Grid and Generate a Mask
Here, I will recreate the grid I will use in my model and read in the bathymetry file (see previous notebooks for details):

In [48]:
# define the parameters that will be used in the data file
delX = 1/48
delY = 1/48
xgOrigin = -68
ygOrigin = 62
n_rows = 440
n_cols = 960

# recreate the grids that will be used in the model
xc = np.arange(xgOrigin+delX/2, xgOrigin+n_cols*delX, delX)
yc = np.arange(ygOrigin+delY/2, ygOrigin+n_rows*delY+delY//2, delY)
XC, YC = np.meshgrid(xc, yc)
print(np.shape(YC))
# read in the bathymetry file
bathy = np.fromfile(os.path.join(input_dir,'high_res_Davis_Strait_bathymetry.bin'),'>f4').reshape(np.shape(XC))

(440, 960)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/cool/Workspace/MLML/ms274_fall2025/Project/high_res/notebooks/high_res_Davis_Strait_bathymetry.bin'

With an eye toward the interpolation to come next, I will make a mask to determine where the interpolatation will take place. I will create this mask by recreating the `hFac` field for my model using the `hFac` module from the `eccoseas` package: 

In [ ]:
from eccoseas.downscale import hFac

In [ ]:
depth = bathy
delR = np.array([ 10.00, 10.00, 10.00, 10.00, 10.00, 10.00, 10.00, 10.01,
                 10.03, 10.11, 10.32, 10.80, 11.76, 13.42, 16.04, 19.82, 24.85,
                 31.10, 38.42, 46.50, 55.00, 63.50, 71.58, 78.90, 85.15, 90.18,
                 93.96, 96.58, 98.25, 99.25,100.01,101.33,104.56,111.33,122.83,
                 139.09,158.94,180.83,203.55,226.50,249.50,272.50,295.50,318.50,
                 341.50,364.50,387.50,410.50,433.50,456.50,])
Z = np.cumsum(delR)
hFacC = hFac.create_hFacC_grid(depth, delR)

The mask is generated by setting all of the non-zero `hFac` points to 1:

In [ ]:
mask = np.copy(hFacC)
mask[mask>0] = 1

To double check the mask was created as expected, I will plot it in comparison to the bathymetry here:

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
C = plt.pcolormesh(XC, YC, bathy, vmin=-5000, vmax=0, cmap='Blues_r')
plt.colorbar(C, orientation = 'horizontal')
plt.title('Model Bathymetry')

depth_level = 0
plt.subplot(1,2,2)
C = plt.pcolormesh(XC, YC, mask[0], vmin=0, vmax=1, cmap='Greys')
plt.colorbar(C, orientation = 'horizontal')
plt.title('Mask (depth level = '+str(depth_level))

plt.show()

Seems reasonable!

### Step 4: Prepare the grids for interpolation
At this point, we can use the geometry of both grids to check to see which tiles have the information we need. After some trial and error (and referencing the ECCO page), I find that tiles 7 and 10 have the points I need:

In [ ]:
# plot the ECCO tile points from tiles 7 and 10
plt.plot(ecco_XC_tiles[10],ecco_YC_tiles[10],'k.')
plt.plot(ecco_XC_tiles[6],ecco_YC_tiles[6],'k.')

# plot the boundary of the CA model
plt.plot(XC[:,0],YC[:,0], 'g-')
plt.plot(XC[:,-1],YC[:,-1], 'g-')
plt.plot(XC[0,:],YC[0,:], 'g-')
plt.plot(XC[-1,:],YC[-1,:], 'g-')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

As we can see, my model boundary (green) is completely surrounded by the points in tile 8 and 11 (black). I also note that there is extraneous information in points with longitude greater than ~140 - I will omit these points as well. Given these observations, now I read in points from just those tiles to use in interpolation:

In [ ]:
tile_list = [6,10]

# determine the number of points in each set
total_points = 0
for tile_number in tile_list:
    total_points += np.size(ecco_XC_tiles[tile_number])

# make empty arrays to fill in
ecco_XC_points = np.zeros((total_points, ))
ecco_YC_points = np.zeros((total_points, ))
ecco_hFacC_points = np.zeros((np.size(ecco_RF_tiles) , total_points))
ecco_hFacW_points = np.zeros((np.size(ecco_RF_tiles) , total_points))
ecco_hFacS_points = np.zeros((np.size(ecco_RF_tiles) , total_points))
ecco_maskC_points = np.zeros((np.size(ecco_RF_tiles) , total_points))
ecco_maskS_points = np.zeros((np.size(ecco_RF_tiles) , total_points))
ecco_maskW_points = np.zeros((np.size(ecco_RF_tiles) , total_points))

# loop through the tiles and fill in the XC, YC, and mask points for interpolation
points_counted = 0
for tile_number in tile_list:
    tile_N = np.size(ecco_XC_tiles[tile_number])
    
    ecco_XC_points[points_counted:points_counted+tile_N] = ecco_XC_tiles[tile_number].ravel()
    ecco_YC_points[points_counted:points_counted+tile_N] = ecco_YC_tiles[tile_number].ravel()
    
    for k in range(np.size(ecco_RF_tiles)):
        level_hFacC = ecco_hFacC_tiles[k, tile_number, :, :]
        level_hFacW = ecco_hFacW_tiles[k, tile_number, :, :]
        level_hFacS = ecco_hFacS_tiles[k, tile_number, :, :]
        level_mask_C = np.copy(level_hFacC)
        level_mask_C[level_mask_C>0] = 1
        level_mask_W = np.copy(level_hFacW)
        level_mask_W[level_mask_W>0] = 1
        level_mask_S = np.copy(level_hFacS)
        level_mask_S[level_mask_S>0] = 1
        ecco_hFacC_points[k, points_counted:points_counted+tile_N] = level_hFacC.ravel()
        ecco_hFacW_points[k, points_counted:points_counted+tile_N] = level_hFacW.ravel()
        ecco_hFacS_points[k, points_counted:points_counted+tile_N] = level_hFacS.ravel()
        ecco_maskC_points[k,points_counted:points_counted+tile_N] = level_mask_C.ravel()
        ecco_maskW_points[k,points_counted:points_counted+tile_N] = level_mask_W.ravel()
        ecco_maskS_points[k,points_counted:points_counted+tile_N] = level_mask_S.ravel()
    
    points_counted += tile_N

# remove the points with positive longitude
local_indices = ecco_XC_points<0
ecco_maskC_points = ecco_maskC_points[:, local_indices]
ecco_maskS_points = ecco_maskS_points[:, local_indices]
ecco_maskW_points = ecco_maskW_points[:, local_indices]
ecco_hFacC_points = ecco_hFacC_points[:, local_indices]
ecco_hFacW_points = ecco_hFacW_points[:, local_indices]
ecco_hFacS_points = ecco_hFacS_points[:, local_indices]
ecco_YC_points = ecco_YC_points[local_indices]
ecco_XC_points = ecco_XC_points[local_indices]

Next, we'll read in the real data fields and apply the modifications. First, create a dictionary to store the file names:

In [ ]:
# make a file dictionary to loop over

# test dictionary
# file_prefix_dict = {'THETA':'OCEAN_TEMPERATURE_SALINITY_mon_mean'}

# once tested with the above dict, run and comment with this one
file_prefix_dict = {'THETA':'OCEAN_TEMPERATURE_SALINITY_mon_mean',
                    'SALT':'OCEAN_TEMPERATURE_SALINITY_mon_mean',
                    'UVEL':'OCEAN_VELOCITY_mon_mean',
                    'VVEL':'OCEAN_VELOCITY_mon_mean'}

variable_names = list(file_prefix_dict.keys())

Now, read the fields from the same tiles:

In [ ]:
# make a list to hold all of the ECCO grids
BC_grids = []
timesteps = 12 # data is monthly

# loop through each variable to read in the grid
for variable_name in variable_names:
    
    # create a grid of zeros to fill in
    N = 90*90
    BC_grid = np.zeros((timesteps+1, np.size(ecco_RF_tiles), N*len(tile_list)))

    for timestep in range(timesteps):
    
        if 'VEL' in variable_name:
            ds = nc4.Dataset(os.path.join(data_folder,
                                          file_prefix_dict['UVEL']+'_'+
                                          str(year)+'-'+'{:02d}'.format(timestep+1)+
                                         '_ECCO_V4r4_native_llc0090.nc'))
            u_grid = ds.variables['UVEL'][:,:,:,:,:]
            v_grid = ds.variables['VVEL'][:,:,:,:,:]
            ds.close()
        else:
            ds = nc4.Dataset(os.path.join(data_folder,
                                          file_prefix_dict[variable_name]+'_'+
                                          str(year)+'-'+'{:02d}'.format(timestep+1)+
                                         '_ECCO_V4r4_native_llc0090.nc'))
            grid = ds.variables[variable_name][:,:,:,:,:]
            ds.close()
        
        # loop through the tiles
        points_counted = 0
        for tile_number in tile_list:
            if 'VEL' in variable_name: # when using velocity, need to consider the tile rotations
                if variable_name == 'UVEL':
                    for k in range(np.size(ecco_RF_tiles)):
                        BC_grid[timestep, k,points_counted:points_counted+N] = \
                             (AngleCS[tile_number] * u_grid[0,k,tile_number,:,:] \
                              - 1 * AngleSN[tile_number] * v_grid[0, k,tile_number,:,:]).ravel()
                if variable_name == 'VVEL':
                    for k in range(np.size(ecco_RF_tiles)):
                        BC_grid[timestep, k,points_counted:points_counted+N] = \
                             (AngleSN[tile_number] * u_grid[0,k,tile_number,:,:] \
                              + AngleCS[tile_number] * v_grid[0, k,tile_number,:,:]).ravel()          

            # if 'VEL' in variable_name: # when using velocity, need to consider the tile rotations
            #     if variable_name == 'UVEL':
            #         if tile_number<6:
            #             for k in range(np.size(ecco_RF_tiles)):
            #                 BC_grid[timestep, k,points_counted:points_counted+N] = \
            #                      u_grid[0, k, tile_number, :, :].ravel()
            #         else:
            #             for k in range(np.size(ecco_RF_tiles)):
            #                 BC_grid[timestep, k,points_counted:points_counted+N] = \
            #                      v_grid[0, k, tile_number, :, :].ravel()
            #     if variable_name == 'VVEL':
            #         if tile_number<6:
            #             for k in range(np.size(ecco_RF_tiles)):
            #                 BC_grid[timestep, k,points_counted:points_counted+N] = \
            #                      v_grid[0, k, tile_number, :, :].ravel()
            #         else:
            #             for k in range(np.size(ecco_RF_tiles)):
            #                 BC_grid[timestep, k,points_counted:points_counted+N] = \
            #                      -1*u_grid[0, k, tile_number, :, :].ravel()
            else:
                for k in range(np.size(ecco_RF_tiles)):
                    BC_grid[timestep, k,points_counted:points_counted+N] = \
                         grid[0, k, tile_number, :, :].ravel()
            points_counted += N

    
    # remove the points with positive longitudes
    BC_grid = BC_grid[:,:,local_indices]
    #adding an extra timestep to deal with offset 
    BC_grid[12,:,:] = BC_grid[11,:,:]
    
    BC_grids.append(BC_grid)

### Step 5: Interpolate the Fields onto the Model Grid
Next, I will interpolate the ECCO external fields I read in onto my model domain. I will use the `horizonal` module from the `eccoseas` package to accomplish this interpolation.

In [ ]:
# define the boundary list for the model
boundary_list = ['south','north']

In [ ]:
if 'obcs' not in os.listdir(input_dir):
    os.mkdir(os.path.join(input_dir,'obcs'))

In [ ]:
from eccoseas.downscale import horizontal

In [ ]:
# loop through each boundary
for boundary in boundary_list:
    print(' - Creating conditions on the '+str(boundary)+' boundary')

    if boundary == 'west':
        boundary_XC = XC[:,:1]
        boundary_YC = YC[:,:1]
        boundary_mask = mask[:,:,:1]
    elif boundary == 'east':
        boundary_XC = XC[:,-1:]
        boundary_YC = YC[:,-1:]
        boundary_mask = mask[:,:,-1:]
    elif boundary == 'north':
        boundary_XC = XC[-1:,:]
        boundary_YC = YC[-1:,:]
        boundary_mask = mask[:,-1:,:]
    elif boundary == 'south':
        boundary_XC = XC[:1,:]
        boundary_YC = YC[:1,:]
        boundary_mask = mask[:,:1,:]
    else:
        raise ValueError('Boundary '+boundary+' not recognized')

    # loop through each variable and corresponding ECCO grid
    for variable_name, BC_grid in zip(variable_names, BC_grids):

        if variable_name == 'UVEL':
            ecco_mask_points = ecco_maskS_points
        elif variable_name == 'VVEL':
            ecco_mask_points = ecco_maskW_points
        else:
            model_mask = mask
            ecco_mask_points = ecco_maskC_points

        output_grid = np.zeros((timesteps, np.size(delR), np.size(boundary_XC)))
    
        # print a message to keep track of which variable we are working on
        print('    - Interpolating the '+variable_name+' grid')

        for t in range(timesteps):
            print('       - Timestep:',t)
            interpolated_grid = horizontal.downscale_3D_points(np.column_stack([ecco_XC_points, ecco_YC_points]),
                                                               BC_grid[t,:,:], ecco_mask_points, 
                                                               boundary_XC, boundary_YC, boundary_mask)
            for k in range(len(delR)):
                output_grid[t,k,:] = interpolated_grid[k,:,:].ravel()
    
        # output the interpolated grid
        output_file = os.path.join(input_dir,'obcs',variable_name+'_'+boundary+'_'+str(year))
        output_grid.ravel('C').astype('>f4').tofile(output_file)

### Step 6: Plotting the Boundary Fields
Now that the fields have been generated, I will plot them to ensure they look as expected. First, I'll generate some metadata for each one:

In [ ]:
meta_dict = {'THETA':[-3, 6, 'turbo', 'm'],
            'SALT':[32, 35, 'viridis', 'm'],
            'UVEL':[-0.1, 0.1, 'seismic', 'm'],
            'VVEL':[-0.1, 0.1, 'seismic', 'm']}

Then, I'll create all of the subplots:

In [ ]:
fig = plt.figure(figsize=(12,12))


plot_counter = 0
for i in range(len(variable_names)):
    variable_name = variable_names[i]
    
    for boundary in boundary_list:
        
        boundary_grid = np.fromfile(os.path.join(input_dir,'obcs',variable_name+'_'+boundary+'_'+str(year)),'>f4')
    
        if boundary in ['west','east']:
            boundary_grid = boundary_grid.reshape((timesteps, np.shape(delR)[0],np.shape(XC)[0]))
            boundary_grid = boundary_grid[0, :, :] # choose just the first timestep for plotting
            if boundary=='west':
                x = YC[:,1]
            if boundary=='east':
                x = YC[:,-1]
        else:
            boundary_grid = boundary_grid.reshape((timesteps, np.shape(delR)[0],np.shape(XC)[1]))
            boundary_grid = boundary_grid[0, :, :] # choose just the first timestep for plotting
            if boundary=='north':
                x = XC[-1,:]
            if boundary=='south':
                x = XC[1,:]

        plot_counter += 1
        plt.subplot(len(variable_names),len(boundary_list),plot_counter)
        C = plt.pcolormesh(x, Z, boundary_grid,
                           vmin=meta_dict[variable_names[i]][0],
                           vmax=meta_dict[variable_names[i]][1],
                           cmap=meta_dict[variable_names[i]][2])
        plt.colorbar(C,fraction=0.26)
        plt.gca().invert_yaxis()
    
        if plot_counter%3==1:
            plt.ylabel(variable_name)
        if plot_counter<4:
            plt.title(boundary)

plt.tight_layout()

plt.show()

Looks good! Now, with the initial conditions, external forcing conditions, and boundary conditions we are nearly ready to start testing the model!

### Step 7: Run-time considerations
To use the grids as boundary conditions in the model, I will use the `obcs` package - short for Open Boundary Condtions. To ensure the boundary conditions are included in the model, I will add a line for `obcs` in my `code/packages.conf` compile time file.

Next, I will add a line with ` useOBCS=.TRUE.` to the `data.pkg` file and a new file called `data.obcs` to my run directory. See Canvas for an editable `data.obcs` file.